## Setup

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm_config = {"model": "gpt-4o-mini"}

## Creating the needed agents

In [9]:
from autogen import ConversableAgent

onboarding_personal_information_agent = ConversableAgent(
    name="OnboardingPersonalInformationAgent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [10]:
onboarding_topic_preference_agent = ConversableAgent(
    name="OnboardingTopicpreferenceAgent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [11]:
customer_engagement_agent = ConversableAgent(
    name="CustomerEngagementAgent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [12]:
customer_proxy_agent = ConversableAgent(
    name="customerproxyagent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [13]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

In [14]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
OnboardingPersonalInformationAgent (to customerproxyagent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------
customerproxyagent (to OnboardingPersonalInformationAgent):

Faiz

--------------------------------------------------------------------------------
OnboardingPersonalInformationAgent (to customerproxyagent):

Great to meet you, Faiz! Could you please share your location?

--------------------------------------------------------------------------------
customerproxyagent (to OnboardingPersonalInformationAgent):

Lahore

--------------------------------------------------------------------------------

*****************************************************************

In [15]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
  "name": "Faiz",
  "location": "Lahore"
}
```


Faiz, from Lahore, is interested in reading about the topic of fear of losing love.


Faiz is interested in reading about the fear of losing love. The conversation provided a fun fact about the psychological origins of this fear, a short story illustrating how sharing insecurities can strengthen relationships, and a light-hearted joke to bring humor to the topic. The overall takeaway emphasizes that embracing humor and open communication can help alleviate fears and foster love.




In [18]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 5.5499999999999994e-05, 'gpt-4o-mini-2024-07-18': {'cost': 5.5499999999999994e-05, 'prompt_tokens': 210, 'completion_tokens': 40, 'total_tokens': 250}}, 'usage_excluding_cached_inference': {'total_cost': 5.5499999999999994e-05, 'gpt-4o-mini-2024-07-18': {'cost': 5.5499999999999994e-05, 'prompt_tokens': 210, 'completion_tokens': 40, 'total_tokens': 250}}}


{'usage_including_cached_inference': {'total_cost': 2.5199999999999996e-05, 'gpt-4o-mini-2024-07-18': {'cost': 2.5199999999999996e-05, 'prompt_tokens': 88, 'completion_tokens': 20, 'total_tokens': 108}}, 'usage_excluding_cached_inference': {'total_cost': 2.5199999999999996e-05, 'gpt-4o-mini-2024-07-18': {'cost': 2.5199999999999996e-05, 'prompt_tokens': 88, 'completion_tokens': 20, 'total_tokens': 108}}}


{'usage_including_cached_inference': {'total_cost': 0.0003447, 'gpt-4o-mini-2024-07-18': {'cost': 0.0003447, 'prompt_tokens': 582, 'completion_tokens': 429, 'total_tokens': 1011}},

In [17]:
for chat_result in chat_results:
    print(chat_result.chat_history)
    print("\n")

[{'content': "Hello, I'm here to help you get started with our product.Could you tell me your name and location?", 'role': 'assistant', 'name': 'OnboardingPersonalInformationAgent'}, {'content': 'Faiz', 'role': 'user', 'name': 'customerproxyagent'}, {'content': 'Great to meet you, Faiz! Could you please share your location?', 'role': 'assistant', 'name': 'OnboardingPersonalInformationAgent'}, {'content': 'Lahore', 'role': 'user', 'name': 'customerproxyagent'}]


[{'content': 'Great! Could you tell me what topics you are interested in reading about?\nContext: \n```json\n{\n  "name": "Faiz",\n  "location": "Lahore"\n}\n```', 'role': 'assistant', 'name': 'OnboardingTopicpreferenceAgent'}, {'content': 'Fear to lossing love', 'role': 'user', 'name': 'customerproxyagent'}]


[{'content': 'Let\'s find something fun to read.\nContext: \n```json\n{\n  "name": "Faiz",\n  "location": "Lahore"\n}\n```\nFaiz, from Lahore, is interested in reading about the topic of fear of losing love.', 'role': 'a